In [ ]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm

import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'AppleGothic'  # macOS


#Fixed Random Seed  & Setting Hyperparameter
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(42)

LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 50
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")



#Data load
train = pd.read_csv('./train/train.csv')

#Define Model
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=64, num_layers=3, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # (B, output_dim)

#Train
def train_lstm(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc ='Training LSTM'):
        #시계열 정렬
        store_train = group.sort_values('영업일자').copy()
        store_train['영업일자'] = pd.to_datetime(store_train['영업일자'])
        store_train['weekday'] = store_train['영업일자'].dt.dayofweek
        
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량','weekday']
        scaler = MinMaxScaler()
        ##매출수량만 정규화, 요일정보 x
        store_train[['매출수량']] = scaler.fit_transform(store_train[['매출수량']])
        train_vals = store_train[features].values  # shape: (N, 2)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = torch.tensor(X_train).float().to(DEVICE)
        y_train = torch.tensor(y_train).float().to(DEVICE)

        model = MultiOutputLSTM(input_dim=2, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        #loss 저장 리스트
        train_losses = []

        model.train()
        for epoch in range(EPOCHS):
            model.train()
            total_loss = 0  # 전체 에폭의 평균 loss 계산용
            idx = torch.randperm(len(X_train))  # 데이터 섞기
            
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                
                output = model(X_batch)
                loss = criterion(output, y_batch)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            avg_loss = total_loss / (len(X_train) // BATCH_SIZE + 1)
            
            train_losses.append(avg_loss)  # loss 기록

        #loss 시각화
        #visualize_loss(train_losses, store_menu)

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (28, 2)
        }

    return trained_models

def visualize_loss(train_losses, store_menu) :
    # ✅ 시각화 (모델별 loss 그래프)
    plt.plot(train_losses, label='Train Loss')
    plt.title(f"[{store_menu}] Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()
    plt.show()

#Prediction
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        store_test_sorted['영업일자'] = pd.to_datetime(store_test_sorted['영업일자'])
        store_test_sorted['weekday'] = store_test_sorted['영업일자'].dt.dayofweek
        recent_vals = store_test_sorted[['매출수량','weekday']].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        
        # 매출수량만 정규화
        recent_vals[:, 0] = scaler.transform(recent_vals[:, 0].reshape(-1, 1)).squeeze()
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        restored = []
        for i in range(PREDICT):
            val = pred_scaled[i]
            # DataFrame으로 감싸서 inverse_transform
            restored_val = scaler.inverse_transform(pd.DataFrame([[val]], columns=['매출수량']))[0, 0]
            restored.append(max(restored_val, 0))  # 음수 방지


        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu[0],
                '매출수량': val
            })
    

    return pd.DataFrame(results)

def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량']
    ))

    final_df = sample_submission.copy()

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

    return final_df


In [30]:
# 학습
trained_models = train_lstm(train)

Training LSTM: 100%|██████████| 193/193 [27:02<00:00,  8.41s/it]


In [31]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('./test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)
    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)
    
full_pred_df = pd.concat(all_preds, ignore_index=True)

/Users/widohun/tf_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/widohun/tf_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/widohun/tf_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/widohun/tf_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/widohun/tf_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/User

In [33]:
sample_submission = pd.read_csv('./sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv('2nd.csv', index=False, encoding='utf-8-sig')
result = pd.read_csv('2nd.csv')
display(result.head())

/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_7453/1683635868.py:189: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.4571375846862793' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_7453/1683635868.py:189: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.9221973419189453' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_7453/1683635868.py:189: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.6678698062896729' has dtyp

,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,2.457138,0.000000,3.922197,1.667870,0.711641,0.354252,0.000000,0.728236,0.688876,...,0.152260,3.293851,6.554350,3.910830,40.871803,5.544255,4.338501,9.245001,2.517511,9.648119
1,TEST_00+2일,1.073679,12.144321,3.038941,1.055035,0.039878,0.163606,0.000000,0.201153,0.514061,...,0.000000,0.483210,0.596791,0.067247,25.422882,1.405562,1.255057,9.250990,2.394780,6.507873
2,TEST_00+3일,0.000000,13.851008,1.547581,0.904941,0.124671,0.000000,0.000000,0.139471,0.547331,...,0.000000,0.000000,2.306186,0.000000,18.705839,0.000000,0.000000,2.265581,0.032749,3.179231
3,TEST_00+4일,0.561959,25.470930,1.773850,1.260938,0.214548,0.172710,0.697689,0.328657,1.119696,...,0.000000,0.000000,2.088674,0.000000,22.657549,2.749539,0.000000,0.687461,0.000000,1.996862
4,TEST_00+5일,1.497928,62.364643,2.503058,1.544045,0.267617,0.406990,4.689624,0.432974,0.762773,...,0.455268,0.411865,1.625148,0.898392,24.823866,4.694133,3.301718,10.601496,2.336352,5.177643
